In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame  # or: from pyspark.sql.dataframe import DataFrame
from pyspark.sql import functions as F, types as T

spark = SparkSession.builder.appName("check").getOrCreate() # type: ignore # ignore

df: DataFrame = (
    spark.read.format("csv")
    .option("inferSchema", True)
    .option("header", True)
    .load("fk.csv")
)
df.show()

+------------+-------+------------+--------------------+------------+--------------------+-------+---------+------+------+-----+-----+-----+-----+
|  PORTFOLIOS|ADS TYP|ACCOUNT NAME|           CAMPAIGNS| ad_group_id|             KEYWORD|   DATE|MATCH TYP|IMPRSN|CLICKS|UNITS|  CPC|SALES|SPEND|
+------------+-------+------------+--------------------+------------+--------------------+-------+---------+------+------+-----+-----+-----+-----+
| Chimney-HOB|    PCA|   BeyondNxt|PCA-Spotlight-Ast...|M4FUE3IXDGOD|best chimney for ...|07-2025|    EXACT|   331|     7|    0| 33.0|    0|232.0|
| Chimney-HOB|    PCA|   BeyondNxt|  PCA-Spotlight-Cube|9W76JZTQH4AN|       slant chimney|07-2025|    EXACT|   189|     5|    0| 42.0|    0|208.0|
| Chimney-HOB|    PCA|   BeyondNxt|PCA-Spotlight-Ast...|M4FUE3IXDGOD|pigeon chimney fo...|07-2025|    EXACT|   435|     5|    0| 91.0|    0|457.0|
| Chimney-HOB|    PCA|   BeyondNxt|  PCA-Spotlight-Cube|9W76JZTQH4AN|kitchen chimney a...|07-2025|    EXACT|    41|   

In [10]:
try: spark.stop()
except: pass

from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("s3-read")
    .config("spark.jars.packages",
            "org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.12.262")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.aws.credentials.provider",
            "com.amazonaws.auth.DefaultAWSCredentialsProviderChain")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    # optional if bucket is in ap-south-1
    .config("spark.hadoop.fs.s3a.endpoint", "s3.ap-south-1.amazonaws.com")
    .getOrCreate()
)


In [11]:
df = (spark.read.format("csv")
      .option("header", True)
      .option("inferSchema", True)
      .load("s3a://mkdv1/brands.csv"))

df.show(5)

Py4JJavaError: An error occurred while calling o225.load.
: java.lang.RuntimeException: java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.S3AFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2688)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3431)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.datasources.DataSource$.$anonfun$checkAndGlobPathIfNecessary$1(DataSource.scala:724)
	at scala.collection.immutable.List.map(List.scala:293)
	at org.apache.spark.sql.execution.datasources.DataSource$.checkAndGlobPathIfNecessary(D

In [9]:
# 1) Did Spark pick up the packages flag?
print("packages:", spark.sparkContext.getConf().get("spark.jars.packages", None))

# 2) Is the S3A class actually present?
try:
    spark.sparkContext._jvm.java.lang.Class.forName("org.apache.hadoop.fs.s3a.S3AFileSystem")
    print("S3A jar present ✅")
except Exception as e:
    print("S3A jar missing ❌", e)

packages: org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.12.262
S3A jar missing ❌ An error occurred while calling z:java.lang.Class.forName.
: java.lang.ClassNotFoundException: org.apache.hadoop.fs.s3a.S3AFileSystem
	at java.base/jdk.internal.loader.BuiltinClassLoader.loadClass(BuiltinClassLoader.java:641)
	at java.base/jdk.internal.loader.ClassLoaders$AppClassLoader.loadClass(ClassLoaders.java:188)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:525)
	at java.base/java.lang.Class.forName0(Native Method)
	at java.base/java.lang.Class.forName(Class.java:375)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:77)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:568)
	at py4j.reflection.MethodInvoker.invoke(

In [13]:
import os
# Add the S3 connector jars via packages
os.environ["PYSPARK_SUBMIT_ARGS"] = (
    "--packages org.apache.hadoop:hadoop-aws:3.3.4,"
    "com.amazonaws:aws-java-sdk-bundle:1.12.262 pyspark-shell"
)

# (Re)start Spark cleanly
try: spark.stop()
except: pass

from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("s3-read")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.aws.credentials.provider",
            "com.amazonaws.auth.DefaultAWSCredentialsProviderChain")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    # Helpful for ap-south-1
    .config("spark.hadoop.fs.s3a.endpoint", "s3.ap-south-1.amazonaws.com")
    .getOrCreate()
)

# Verify jars are present
print("packages:", spark.sparkContext.getConf().get("spark.jars.packages"))
spark.sparkContext._jvm.java.lang.Class.forName("org.apache.hadoop.fs.s3a.S3AFileSystem")
print("S3A ready ✅")


packages: None


Py4JJavaError: An error occurred while calling z:java.lang.Class.forName.
: java.lang.ClassNotFoundException: org.apache.hadoop.fs.s3a.S3AFileSystem
	at java.base/jdk.internal.loader.BuiltinClassLoader.loadClass(BuiltinClassLoader.java:641)
	at java.base/jdk.internal.loader.ClassLoaders$AppClassLoader.loadClass(ClassLoaders.java:188)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:525)
	at java.base/java.lang.Class.forName0(Native Method)
	at java.base/java.lang.Class.forName(Class.java:375)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:77)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:568)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:3